# What is an Agent?

## Three competing definitions

1. AI systems that can do work for you independently - Sam Altman

2. A system in which an LLM controls the workflow - Anthropic

3. An LLM agent runs tools in a loop to achieve a goal

## The third one is the new, emerging definition

But what does it mean?

Let's make it real.

## First up, to demystify 3 concepts:

1. Tools - allow an LLM to call a function we write, perhaps to do math, query a database or call an API. Sounds spooky? It's another illusion. LLMs only generate tokens - they can't call functions!

2. Structured outputs - require an LLM to respond in JSON according to a particular JSON spec

3. Agent Framework - useful code which simplifies (1) and (2), amongst other things, making it easier to connect together LLM calls


In [21]:
from agents import Agent, Runner, function_tool, set_default_openai_client
from dotenv import load_dotenv
from rich.console import Console
from openai import OpenAI
import asyncio
from openai import AsyncOpenAI

load_dotenv(override=True)

True

In [17]:
import os

openrouter_url = "https://openrouter.ai/api/v1/"
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
openrouter_client = OpenAI(api_key=openrouter_api_key, base_url=openrouter_url)
set_default_openai_client(openrouter_client)


In [23]:
async def main():
    agent = Agent("Joke teller", model="openai/gpt-4.1-mini")
    result = await Runner.run(agent, "Tell me a joke for some data scientists")
    print(result.final_output)

In [25]:
await main()

Error getting response: object Response can't be used in 'await' expression. (request_id: None)


TypeError: object Response can't be used in 'await' expression

In [28]:
import os
import asyncio
from openai import AsyncOpenAI
from agents import Agent, Runner, set_default_openai_client

# Import the actual runner instance the library uses
from agents.run import DEFAULT_AGENT_RUNNER
from dotenv import load_dotenv

load_dotenv(override=True)


async def main():
    # 1. Setup the Async OpenRouter client
    openrouter_client = AsyncOpenAI(
        api_key=os.getenv("OPENROUTER_API_KEY"), base_url="https://openrouter.ai/api/v1"
    )

    # 2. Force the library to use this client globally
    set_default_openai_client(openrouter_client)

    # 3. CRITICAL: Manually inject the client into the default runner
    # This bypasses the library's internal logic that causes the TypeError
    DEFAULT_AGENT_RUNNER.client = openrouter_client

    # 4. Define the agent
    agent = Agent("Joke teller", model="openai/gpt-4o-mini")

    # 5. Run it
    result = await Runner.run(agent, "Tell me a joke for some data scientists")

    print(result.final_output)


# Run in Notebook
await main()

Why did the data scientist break up with the statistician?

Because she found him mean!


In [ ]:
agent = Agent("Joke teller", model="gpt-4.1-mini")
result = await Runner.run(agent, "Tell me a joke for some data scientists")
print(result.final_output)

In [ ]:
todos = []
completed = []

In [ ]:
def get_todo_report(print: bool=False) -> str:
    """Get a report of all todos."""
    result = ""
    for index, (todo, complete) in enumerate(zip(todos, completed)):
        check = "X" if complete else " "
        start = "[strike][green]" if complete else ""
        end = "[/strike][/green]" if complete else ""
        result += f"Todo #{index + 1}: [{check}] {start}{todo}{end}\n"
    if print:
        Console().print(result)
    return result

In [ ]:
get_todo_report()

In [ ]:
def create_todos(descriptions: list[str]) -> str:
    """Add new todos from a list of descriptions and return the full list"""
    todos.extend(descriptions)
    completed.extend([False] * len(descriptions))
    return get_todo_report(print=True)

def mark_complete(index: int) -> str:
    """Mark complete the todo at the given position (starting from 1) and return the full list"""
    if 1 <= index <= len(todos):
        completed[index - 1] = True
    else:
        return "No todo at this index."
    return get_todo_report(print=True)

In [ ]:
todos, completed = [], []

create_todos(["Buy groceries", "Finish lab1", "Go for a walk"])

In [ ]:
mark_complete(1)

In [ ]:
@function_tool
def create_todos(descriptions: list[str]) -> str:
    """Add new todos from a list of descriptions and return the full list"""
    todos.extend(descriptions)
    completed.extend([False] * len(descriptions))
    return get_todo_report(print=True)

@function_tool
def mark_complete(index: int, completion_notes: str) -> str:
    """Mark complete the todo at the given position (starting from 1) and return the full list
    
    Args:
        index: The 1-based index of the todo to mark as complete
        completion_notes: Notes about how you completed the todo in rich console markup
    """
    if 1 <= index <= len(todos):
        completed[index - 1] = True
    else:
        return "No todo at this index."
    Console().print(completion_notes)
    return get_todo_report(print=True)

In [ ]:
mark_complete.params_json_schema

In [ ]:
instructions = """
You are given a problem to solve, by using your todo tools to plan a list of steps, then carrying out each step in turn.
Now use the todo list tools, create a plan, carry out the steps, and reply with the solution.
Provide your solution in Rich console markup (e.g. [bold red]Error[/bold red]) to indicate colors and styles.
Do not ask the user questions or clarification; respond only with the answer after using your tools.
"""
tools = [create_todos, mark_complete]
agent = Agent("Puzzle Agent", model="gpt-4.1-mini", instructions=instructions, tools=tools)

In [ ]:
task = "A train leaves Boston at 2:00 pm traveling 60 mph. Another train leaves New York at 3:00 pm traveling 80 mph toward Boston. When do they meet?"
todos, completed = [], []
response = await Runner.run(agent, task)
Console().print("\n\n" + response.final_output)